# 3. Nettoyage des données scrappées et merging avec Data_cars

## Objectifs :
* Réaliser le preprocessing du dataset scrapé depuis le site [Caradisiac](https://www.caradisiac.com/), notamment de la colone caracteristiques (Voir le notebook **2_Collecte_data_externe** pour la méthode de scarping),
* Croisement des données scrappées (caradisiac_dataset) avec le dataset fourni dans le cadre de ce projet (Data_cars.csv)

## Plan :
1. Preprocessing du dataset scrapé
        1.1. Concaténation des datasets scrappés
        1.2. Traitement de la variable'Caracteristiques'
            1.2.1. Convertion vers le format Json
            1.2.2. Récupération de la liste des sous-champs
        1.3. Les variables Prix & time
2. Merging des datasets
        2.1 Création de la variable du merge 'version'
        2.2. Déclaration des fonctions de ngrams et GetNeaestN
        2.3. Matching
        2.4. Merge entre les datasets
3. Prochaines étapes

In [1]:
import pandas as pd
import numpy as np
import re
from collections import ChainMap
import json
import re

pd.set_option('max_colwidth', 10000)
pd.options.display.max_rows = 4000

In [14]:
# Fromatage de l'affichage des dataframe
%%HTML
<style>.dataframe th, td:first-child{background:#3f577c;font-family:monospace;color:white;border:3px solid white;
text-align:left !important;}#codex{float:right;}</style>

## 1. Preprocessing du dataset scrapé

### 1.1. Concaténation des datasets scrappés

Suite à des problèmes de connexion, nous n'avons pas pu récupérer un unique dataset des informations de toutes les versions des vehicules présentes sur le site de Caradisiac. <br>
Ainsi, nous avons récupéré plusieurs datasets que nous allons concatener dans cette section.

In [2]:
import glob

# Chemin vers le dossier contenant les fichiers csv à concatener
# Fichiers contenant les informations des versions présentes sur la premeière page
path = r'../Datasets/Caradisiac'

# Liste de tous les chemins vers les fichiers csv
all_files = glob.glob(path + "/*.csv")

# Chargement des données dans une liste de Dataframes
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concatenation de la liste des dataframes
data_first_pages = pd.concat(li, axis=0, ignore_index=True)

# Chemin vers le dossier contenant les fichiers csv à concatener
# Fichiers contenant les informations des versions présentes sur les pages suivantes
path = r'../Datasets/Caradisiac/next_pages'

# Liste de tous les chemins vers les fichiers csv
all_files = glob.glob(path + "/*.csv")

# Chargement des données dans une liste de Dataframes
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
# Concatenation de la liste des dataframes
data_next_pages = pd.concat(li, axis=0, ignore_index=True)

In [8]:
data_first_pages.shape, data_next_pages.shape

((123597, 17), (15450, 18))

In [3]:
# Suppression des colonnes inutiles
data_first_pages.drop(columns=["Unnamed: 0",
                               "Unnamed: 0.1", "model_gamme_url", "model_url", "fich_tech_href", 
                              "year_url", "version_url"], inplace=True)
data_next_pages.drop(columns=["Unnamed: 0", 
                              "Unnamed: 0.1", "model_gamme_url", "model_url", "fich_tech_href", 
                              "year_url", "version_url", "next_page"], inplace=True)

In [10]:
data_first_pages.shape, data_next_pages.shape

((123597, 10), (15450, 10))

In [4]:
# Concatenation des deux dataframes contenant les informations sur 
#les versions présentes sur les premières page avec celles présentes dans les pages suivantes
caradisiac_dataset = pd.concat([data_first_pages, data_next_pages], axis=0, ignore_index=True)
caradisiac_dataset.shape

(139047, 10)

In [15]:
caradisiac_dataset.head(1)

,brand,time,model_gamme,model,year,version_name,prix,caracteristiques,equips_all,options
0,hyundai,Les modèles d'aujourd'hui,hyundai kona,hyundai kona,2018,1.0 T-GDI 120 EXECUTIVE,25 900 €,"[{'Date de commercialisation': '01/09/2017'}, {'Date de fin de commercialisation': '15/04/2018'}, {'Durée de la garantie': '60 mois (kilométrage illimité)'}, {'Intervalles de révision en km': 'NC'}, {'Intervalles de révision maxi': 'NC'}, {'Longueur': '4,16 m'}, {'Largeur sans rétros': '1,80 m'}, {'Hauteur': '1,56 m'}, {'Empattement': '2,60 m'}, {'Volume de coffre mini/maxi': '361 l / 1 143 l'}, {'Nombre de portes': '5'}, {'Nombre de places assises': '5'}, {'Poids à vide': '1 308 kg'}, {'Motorisation': 'Essence'}, {'Puissance fiscale': '6 CV'}, {'Moteur': ' 3 cylindres en ligne, 12 soupapes'}, {'Cylindrée': '998 cm3'}, {'Puissance': '120 ch à 6 000 trs/min'}, {'Couple': '172 Nm à 1 500 trs/min'}, {'Type de suralimentation': 'turbo'}, {'Boîte de vitesse, Nb de rapports': 'Mécanique à 6 rapports'}, {'Roues motrices': 'AV'}, {'Vitesse maxi': '181 km/h'}, {'0 à 100 km/h': '12 sec'}, {'Consommation urbaine': '6,30 l / 100 km'}, {'Consommation extra-urbaine': '5 l / 100 km'}, {'Consommation mixte': '5,40 l / 100 km'}, {'Volume du réservoir': '50 l'}, {'Emission de CO2 (NEDC)': '125 g/km'}, {'Bonus / Malus écologique': '--'}, {'NormeEuro': 'EU6'}, {'Suspension avant': 'jambe de force'}, {'Suspension arrière': 'essieu torsionnel'}, {'Taille des pneus avant': '235/45/18'}, {'Taille des pneus arrière': '235/45/18'}, {'Roue de secours': 'NC'}, {'Freins à disque': '4'}, {'Diamètre de braquage entre murs': 'NC'}, {'Diamètre de braquage entre trottoirs': '10,60 m'}, {""Angle d'attaque"": 'NC'}, {'Angle de fuite': 'NC'}, {'Blocage de différentiel': 'non'}]","[{'Equipements de sécurité': ['6 airbags', 'ESP', 'Airbags rideaux', 'Système détection de collision', 'Indicateur de sous-gonflage des pneus', ""Assistance au freinage d'urgence"", 'Kit anticrevaison', 'Essuie glace capteur de pluie', 'Antipatinage', ""Capteur d'angle mort"", 'ABS', 'Avertisseur de franchissement de ligne', 'Airbags latéraux', 'Aide au démarrage en côte']}, {'Equipements intérieur': ['Système audio écran tactile', 'Bouton démarrage', 'Air conditionné auto', 'Système de navigation', 'Bluetooth inclut musique en streaming, connexion téléphone', 'Régulateur de vitesse', 'Smart card / Smart key', 'Volant cuir', 'Aide au stationnement arrière', 'Tableau de bord vision tête haute', 'Ordinateur de bord', 'Siège avant électrique', 'Lecteur MP3', 'Taille écran navigation 8 pouces', 'Accoudoir central avant', 'Système de navigation info trafic', 'Limiteur de vitesse', 'Rétroviseur jour/nuit', 'Volant multi-fonction', 'Bluetooth', '8 haut-parleurs', 'Air conditionné 1 zones', 'Siège avant chauffant', 'Réglage du volant en hauteur, en profondeur', 'Taille écran multi-fonctions 8 pouces']}, {'Equipements extérieur et chassis': ['Phares à allumage automatique', 'Ampoules de phares LED', 'Feux arrières à LED', 'Antibrouillards avant', 'Rétroviseurs extérieurs réglage électrique', 'Feux de croisement à LED', 'Rétroviseurs extérieurs chauffants', 'Feux de croisement automatiques', 'Rétroviseurs rabattables électriquement']}]","[{'Peinture métallisée': '550 €'}, {'Peinture métallisée bi-ton': '950 €'}, {'Peinture métallisée Velvet Dune': '0 €'}, {'Roue de secours temporaire': '100 €'}]"


In [16]:
caradisiac_dataset.dtypes

brand               object
time                object
model_gamme         object
model               object
year                 int64
version_name        object
prix                object
caracteristiques    object
equips_all          object
options             object
dtype: object

## 1.2. Traitement de la variable 'Caracteristiques'

In [18]:
# Visialisation d'un échantillon
caradisiac_dataset['caracteristiques'].head(1)

0    [{'Date de commercialisation': '01/09/2017'}, {'Date de fin de commercialisation': '15/04/2018'}, {'Durée de la garantie': '60 mois (kilométrage illimité)'}, {'Intervalles de révision en km': 'NC'}, {'Intervalles de révision maxi': 'NC'}, {'Longueur': '4,16 m'}, {'Largeur sans rétros': '1,80 m'}, {'Hauteur': '1,56 m'}, {'Empattement': '2,60 m'}, {'Volume de coffre mini/maxi': '361 l / 1 143 l'}, {'Nombre de portes': '5'}, {'Nombre de places assises': '5'}, {'Poids à vide': '1 308 kg'}, {'Motorisation': 'Essence'}, {'Puissance fiscale': '6 CV'}, {'Moteur': ' 3 cylindres en ligne, 12 soupapes'}, {'Cylindrée': '998 cm3'}, {'Puissance': '120 ch à 6 000 trs/min'}, {'Couple': '172 Nm à 1 500 trs/min'}, {'Type de suralimentation': 'turbo'}, {'Boîte de vitesse, Nb de rapports': 'Mécanique à 6 rapports'}, {'Roues motrices': 'AV'}, {'Vitesse maxi': '181 km/h'}, {'0 à 100 km/h': '12 sec'}, {'Consommation urbaine': '6,30 l / 100 km'}, {'Consommation extra-urbaine': '5 l / 100 km'}, {'Consomma

La variable 'Caracteristiques' est sous forme d'une liste de dictionnaires représentées comme des Objets (strings) sous le dataframe.
Ainsi, il faut transformer cette variable vers le format Json afin de faciliter sa manipulation. <br>
**Le but est de spliter la variable *caracteristiques* en plusieurs variables telles que : Date de commercialisation, Date de fin de commercialisatin, Puissance, Motorisation, Nombre de places assises, Emission de CO2 ....etc.**

In [548]:
# Comptage du nombre de versions qui contiennent des champs avec "\\xa".
# Il faut remplacer "\\xa" par le vide car cela crée des erreurs lors de la conversion vers le format Json. 
# problème du backslash

#III (2) SPORTBACK 40 E-TRON DESIGN LUXE S TRONIC
#III (2) SPORTBACK 40 E-TRON BUSINESS LINE S TRONIC
# .... etc.
caradisiac_dataset_dict = caradisiac_dataset.to_dict(orient='records')
delete = []
for elm in caradisiac_dataset_dict:
    if elm['caracteristiques'].find("\\xa")!=-1:
        delete.append(elm['version_name'])
len(delete)

1111

### 1.2.1. Convertion vers le format Json

In [5]:
# Remplacer les ' par des " afin que la conversion vers le Json soit possible
caradisiac_dataset['caracteristiques'] = caradisiac_dataset['caracteristiques'].str.replace(r'{\'', '{"',  regex=True)#.replace("[", "").replace("]","")
caradisiac_dataset['caracteristiques'] = caradisiac_dataset['caracteristiques'].str.replace(r'\'}', '"}', regex=True)#.replace("[", "").replace("]","")

caradisiac_dataset['caracteristiques'] = caradisiac_dataset['caracteristiques'].str.replace(r' \'', ' "',  regex=True)#.replace("[", "").replace("]","")
caradisiac_dataset['caracteristiques'] = caradisiac_dataset['caracteristiques'].str.replace(r'\':', '":',  regex=True)#.replace("[", "").replace("]","")

# Remplacer "\\xa" par le vide car cela crée des erreurs lors de la conversion vers le format Json  
# à cause des backslash
caradisiac_dataset['caracteristiques'] = caradisiac_dataset['caracteristiques'].str.replace(r'\\xa0', '',  regex=True)#.replace("[", "").replace("]","")

# Conversion vers Json
caradisiac_dataset['caracteristiques'] = caradisiac_dataset.caracteristiques.apply(json.loads)

### 1.2.2. Récupération de la liste des sous-champs

In [23]:
# Conversion du dataset au format dictionnaire
caradisiac_dataset_dict = caradisiac_dataset.to_dict(orient='records')

keys_all = []
i= 0

# récuperer toutes les keys du champs 'caracteristiques'
for elm in caradisiac_dataset_dict:
    keys = (dict(ChainMap(*elm['caracteristiques']))).keys()
    keys_all.append(list(keys))
    
keys_all = np.concatenate(keys_all).ravel().tolist()
keys_all = set(keys_all)

In [27]:
print("Il y a %d variables \n"% (len(keys_all)))
print("Liste des variables: ")
print(keys_all)

Il y a 55 variables 

Liste des variables: 
{"Angle d'attaque", 'Consommation extra-urbaine', 'Volume de coffre mini/maxi', 'Taille des pneus arrière', 'Type de suralimentation', 'Position', 'Roue de secours', 'Roues motrices', 'Poids à vide', 'Location batterie', 'Caractéristiques moteur N°1', 'Date de fin de commercialisation', 'Norme Euro', 'Date de commercialisation', 'Intervalles de révision maxi', 'Consommation urbaine', 'Cylindrée', 'Taille des pneus avant', 'Direction', 'Angle de fuite', 'Emission de CO2 (NEDC)', 'Caractéristiques moteur N°2', 'Suspension arrière', 'Couple cumulé', 'Consommation mixte', 'Vitesse maxi', 'Largeur sans rétros', 'Boîte de vitesse, Nb de rapports', 'Suspension avant', 'Blocage de différentiel', 'Durée de la garantie', 'Type batterie', 'Durée de charge', 'Diamètre de braquage entre murs', 'Nombre de places assises', 'NormeEuro', 'Moteur', 'Freins à disque', 'Autonomie en electrique', 'Volume du réservoir', 'Empattement', 'Capacité batterie', 'Nombre 

In [6]:
# Dictionnaire contenant les varibales présentes dans le champs 'caracteristiques' avec un renommage
car_cols={'Blocage de différentiel' :  'blockage_differentiel',
'Diamètre de braquage entre trottoirs' :  'diametre_braquage_entre_trottoirs',
'Diamètre de braquage entre murs' :  'diametre_braquage_entre_murs',
'Freins à disque' : 'freins_disque' ,
'Roue de secours' :  'Roue_secours',
'Taille des pneus arrière' :  'Taille_pneus_arriere',
'Taille des pneus avant' :  'Taille_pneus_avant',
'Suspension arrière' :  'Suspension_arriere',
'Suspension avant' :  'Suspension_avant',
'NormeEuro' :  'Norme_euro',
'Bonus / Malus écologique' :  'Bonus_Malus_ecologique',
'Emission de CO2 (NEDC)' :  'Emission_CO2_NEDC',
'Volume du réservoir' :  'Volume_reservoir',
'Consommation mixte' :  'Consommation_mixte',
'Consommation extra-urbaine' :  'Consommation_extra_urbaine',
'Consommation urbaine' :  'consommation_urbaine',
'0 à 100 km/h' :  '0_a_100km_par_h',
'Vitesse maxi' :  'Vitesse_maxi',
'Roues motrices' :  'Roues_motrices',
'Boîte de vitesse, Nb de rapports' :  'Boite_vitesse_Nb_rapports',
'Type de suralimentation' :  'Type_suralimentation',
'Couple' :  'Couple',
'Puissance' :  'Puissance',
'Cylindrée' :  'Cylindree',
'Moteur' :  'Moteur',
'Puissance fiscale' :  'Puissance_fiscale',
'Motorisation' :  'Motorisation',
'Poids à vide' :  'Poids_a_vide',
'Nombre de places assises' :  'Nb_places_assises',
'Nombre de portes' :  'Nombre_portes',
'Volume de coffre mini/maxi' :  'Volume_coffre_mini/maxi' ,
'Empattement' :  'Empattement',
'Hauteur' :  'Hauteur',
'Largeur sans rétros' :  'Largeur_sans_retros',
'Longueur' :  'Longueur',
'Intervalles de révision maxi' :  'Intervalles_revision_maxi',
'Intervalles de révision en km' :  'Intervalles_revision_km',
'Durée de la garantie' :  'Duree_garantie',
'Date de fin de commercialisation' :  'Date_fin_commercialisation',
'Date de commercialisation' :  'Date_commercialisation',
'Angle dattaque' : 'Angle_attaque',
'Angle de fuite' : 'Angle_ fuite',
'Autonomie en electrique' : 'Autonomie_electrique',
'Boîte de vitesse Nb de rapports' :  'Boite_vitesse_Nb_rapports',
'Capacité batterie' : 'Capacite_ batterie',
'Caractéristiques moteur' : 'Caracteristiques_moteur',
'Caractéristiques moteur N°1' : 'Caractéristiques_moteurN_1',
'Caractéristiques moteur N°2' : 'Caractéristiques_moteurN_2',
'Couple cumulé' : 'Couple_cumule',
'Direction' : 'Direction',
'Durée de charge' : 'Duree_ charge',
'Location batterie' : 'Location_ batterie',
'Nombre de moteurs' : 'Nombre_moteurs',
'Norme Euro' :  'Norme_euro',
'Position' : 'Position',
'Type batterie' : 'Type_batterie'}

In [ ]:
# Création des variables aprtir du champs 'caracteristiques'
for key, value in car_cols.items():
    caradisiac_dataset[value] = caradisiac_dataset.apply(lambda row: dict(ChainMap(*row['caracteristiques'])).get(key), axis=1)
    
# Suppression de la variable 'caracteristiques'
caradisiac_dataset.drop(columns="caracteristiques", inplace=True)

In [605]:
caradisiac_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139047 entries, 0 to 139046
Data columns (total 63 columns):
brand                                139047 non-null object
time                                 139047 non-null object
model_gamme                          139047 non-null object
model                                139047 non-null object
year                                 139047 non-null int64
version_name                         139047 non-null object
prix                                 139045 non-null object
equips_all                           139047 non-null object
options                              139047 non-null object
blockage_differentiel                139045 non-null object
diametre_braquage_entre_trottoirs    139045 non-null object
diametre_braquage_entre_murs         139045 non-null object
freins_disque                        139045 non-null object
Roue_secours                         139045 non-null object
Taille_pneus_arriere                 139045 non-nu

## 1.3. Les variables Prix & time

In [602]:
caradisiac_dataset['prix'] = caradisiac_dataset['prix'].str.replace('€', '').str.replace(' ', '')

caradisiac_dataset.loc[caradisiac_dataset['time']=="Les modèles d'aujourd'hui", 'time'] = "aujourdhui"
caradisiac_dataset.loc[caradisiac_dataset['time']=="Les modèles d'aujourd'hui", 'time'] = "hier"
caradisiac_dataset.loc[caradisiac_dataset['time']=="Les modèles d'aujourd'hui", 'time'] = "autre"

### Save

In [609]:
all_pages.to_csv("Car_versions_all.csv")

## 2. Merging des datasets

Pour le merge, nous avons essayé d'utiliser la librarie FuzzyWuzzy. Mais, elle est très chronophage vu les tailles de nos datasets. <br>
Ainsi, nous avons utilisé une méthode de NLP basée sur TF-IDF décrite dans [ici](https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536), 
 (Voir le [code](https://colab.research.google.com/drive/1qhBwDRitrgapNhyaHGxCW8uKK5SWJblW#scrollTo=xo-X_nds97UN&forceEdit=true&sandboxMode=true)).
 
La méthode basée sur TF-IDF a pris plus de 8 heures pour faire le matching. D'après la rérérence cette méthode est approximativement 66 000 fois plus rapide que le FuzzyWuzzy (nos tests sur quelques echantillons ont peut le confirmer)

In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import re

#!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

pd.set_option('max_colwidth', 10000)

In [32]:
datasets_path = r'../Datasets/'
caradisiac_dataset = pd.read_csv(datasets_path+'scraped_Car_versions.csv')
Data_cars = pd.read_csv(datasets_path+'Data_cars.csv')

/anaconda3/envs/Mask_RCNN_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (54,55,56,57,58,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 2.1. Création de la variable du merge 'version'

Pour Data_cars, on extrait d'abord la variable ***version*** du **Description**. Puis, on concatène les variables :  **Make**, **Model** et **version** afin d'avoir en sortie : **make_model_version**,<br>
Pour caradisiac_dataset, on concatène les variables : **model** et **version_name** afin d'avoir en sortie: **model_version**,<br>

==> Ainsi, nous obtenons les variables **make_model_version** et **model_version** qui sont assez homogènes pour réaliser le matching entre les deux datasets.

In [34]:
Data_cars['version'] = Data_cars['Description'].str.extract(r'(\bversion\b.*?,)', expand=True)       
Data_cars['version'] = Data_cars['version'].str.replace("version: ", "")
Data_cars['version'] = Data_cars['version'].str.replace(",", "")

Data_cars['make_model_version'] = Data_cars['Make'] + " " + Data_cars['Model'] + " " + Data_cars['version']

caradisiac_dataset['model_version'] = caradisiac_dataset['model'] + " " +caradisiac_dataset['version_name']

# Suppression des modèles qui n'existent pas dans le dataframe 'data' (afin de réduire la taille du 'scraped_data')
makes = Data_cars['Make'].unique()
caradisiac_dataset_makes = caradisiac_dataset.loc[caradisiac_dataset['brand'].str.upper().isin(makes)]

In [35]:
Data_cars[['Make', 'Model','version', 'make_model_version']].head(1)

,Make,Model,version,make_model_version
0,PEUGEOT,308,1.6 HDI FAP 92CH BUSINESS 5P,PEUGEOT 308 1.6 HDI FAP 92CH BUSINESS 5P


In [36]:
caradisiac_dataset_makes[['model', 'version_name' ,'model_version']].head(1)

,model,version_name,model_version
0,abarth 124,II 1.4 TURBO 170,abarth 124 II 1.4 TURBO 170


### 2.2. Déclaration des fonctions de ngrams et GetNeaestN
Voir [ici](https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536) pour plus de détails.

In [ ]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = ["|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

###matching query:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

### 2.3. Matching
Création d'une dataframe qui match entre **make_model_version** du Data_cars et **model_version** du caradisiac_dataset. Cette dataframe va servir d'intermediaire pour le merge entre Data_cars et caradisiac_dataset.

In [ ]:
# Vecorisation des données 
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
model_version_scraping = scraped_data_makes['model_version'].unique()
#org_names = data_test['make_model_version'].unique()
tfidf = vectorizer.fit_transform(model_version_scraping)

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

In [ ]:
unique_org = data['make_model_version'].values # colone à matcher avec 'model_version' du 'scraped_data'

# réalisation du neast N
distances, indices = getNearestN(unique_org)

unique_org = list(unique_org) #need to convert back to a list

# recherche des matches
matches = []

for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), scraped_data_makes.values[j][0][64],unique_org[i]]
    matches.append(temp)

# Construction du dataframe des matches 
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','scraped_version','Origional_data'])

**Plus de 8 heures d'exécution**

In [ ]:
# Sauvegarde du matches
matches.to_csv(datasets_path+'matches.csv')

In [39]:
matches.head(5)

,Unnamed: 0,Match confidence (lower is better),scraped_version,Origional_data
0,0,0.76,peugeot 308 1.6 HDI 92 BUSINESS 5P,PEUGEOT 308 1.6 HDI FAP 92CH BUSINESS 5P
1,1,0.33,bmw x6 f16 (F16) XDRIVE30D 258 M SPORT BVA8,BMW X6 (F16) XDRIVE30D 258 M SPORT BVA8
2,2,0.48,audi rs6 (3e generation) avant III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC,AUDI RS6 III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC
3,3,0.00,ford ka 1.3 70 FUN,FORD KA 1.3 70 FUN
4,4,0.46,fiat tipo 2 sw II SW 1.4 95 POP,FIAT TIPO II SW 1.4 95 POP


#### Quelques vérifications

In [40]:
matches['scraped_version'][166690]

'audi a4 (4e generation) avant IV (2) AVANT 2.0 TDI 177 BUSINESS LINE QUATTRO S TRONIC'

In [41]:
matches['Origional_data'][166690]

'AUDI A4 IV (2) AVANT 2.0 TDI 177 BUSINESS LINE QUATTRO S TRONIC'

Le matching est bien réalisé

#### Suppression des doublons dans le match

In [50]:
matches_unique = matches.drop_duplicates()

### 2.4. Merge entre les datasets
Nous utilisons le datasets **matches_unique** pour lier entre **Data_cars** et **caradisiac_dataset_makes**.

In [44]:
# 1er merge
first_merge = pd.merge(Data_cars, matches_unique, left_on='make_model_version', right_on='Origional_data')

In [ ]:
## 2eme merge
second_merge = pd.merge(first_merge, caradisiac_dataset_makes, left_on='scraped_version', right_on='model_version')

#### Suppression des colonnes redondantes

In [ ]:
# Colonnes à supprimer
col_del = ['Match confidence (lower is better)', 'scraped_version', 'Origional_data','make_model_version',
'Unnamed: 0', 'model_version', 'year', 'brand', 'model']

second_merge = second_merge.drop(columns=col_del)

In [ ]:
# Colonnes à renommer
col_rename = {'version_name':'version_name_scr', 'Price':'prix_occasion', 'prix':'prix_neuf'}
data_merged = second_merge.rename(columns=col_rename)

#### Save

In [ ]:
data_merged.to_csv('../Datasets/Data_merged.csv')

## 3. Prochaines étapes
* Exploration du datasets final pour améliorer les performances du modèle de prediction basée sur le RandomForest du notebook **1_DataPreExploration**